In [18]:
!pip install pymongo faker

In [19]:
!pip install faker

In [64]:
from pymongo import MongoClient
from faker import Faker
from bson import ObjectId
from datetime import datetime, timedelta
import random
import json

fake = Faker()

In [36]:
from pymongo import MongoClient

# connect to mongoDB server
client = MongoClient("mongodb://localhost:27017/")
db = client["test_db"]
collection = db["employees_collection"]

In [23]:
from bson import ObjectId

In [24]:
# Predefined sample data

fake = Faker()

from datetime import datetime, timedelta


departments = ["Engineering", "Sales", "Marketing", "HR", "Finance", "Support", "Product"]
positions = ["Junior Developer", "Senior Developer", "Team Lead", "Manager", "DevOps Engineer", "Data Analyst"]
skills_pool = ["Python", "JavaScript", "Java", "C#", "MongoDB", "SQL", "AWS", "Docker", "Kubernetes", "React", "Node.js"]

# Function to generate a single employee document
def generate_employee(emp_num):
    #faker generate a datetime.date so covert to datetime.datetime
    hire_date = datetime.strptime(str(fake.date_between(start_date='-10y', end_date='-1y')),"%Y-%m-%d")
    last_promotion = hire_date + timedelta(days=random.randint(365, 1095))  # 1 to 3 years later

    return {
        "employee_id": f"EMP{emp_num:04d}",
        "first_name": fake.first_name(),
        "last_name": fake.last_name(),
        "email": fake.email().replace('@', '@company.com'),
        "department": random.choice(departments),
        "position": random.choice(positions),
        "salary": random.randint(50000, 180000),
        "years_experience": random.randint(1, 20),
        "performance_rating": round(random.uniform(1.0, 5.0), 1),
        "skills": random.sample(skills_pool, k=random.randint(3, 6)),
        "hire_date": hire_date,
        "last_promotion": last_promotion,
        "is_remote": random.choice([True, False]),
        "address": {
            "city": fake.city(),
            "state": fake.state_abbr(),
            "zip_code": fake.zipcode()
        }
    }

# Generate 10,000 employee documents
employees = [generate_employee(i) for i in range(1, 10001)]

# Optionally: Save to JSON (MongoDB extended JSON format does not support ObjectId/ISODate by default)
# So if you need to dump, you'd need to serialize these types or use pymongo to insert directly.



In [25]:
print(generate_employee(1))

{'employee_id': 'EMP0001', 'first_name': 'Monique', 'last_name': 'Scott', 'email': 'davidhenderson@company.comexample.org', 'department': 'Finance', 'position': 'Senior Developer', 'salary': 126798, 'years_experience': 13, 'performance_rating': 4.8, 'skills': ['MongoDB', 'Kubernetes', 'AWS', 'JavaScript'], 'hire_date': datetime.datetime(2021, 10, 19, 0, 0), 'last_promotion': datetime.datetime(2023, 8, 29, 0, 0), 'is_remote': True, 'address': {'city': 'Natalieville', 'state': 'NM', 'zip_code': '30202'}}


In [26]:
# insert list into my connection

result = collection.insert_many(employees)

print(f"inserted {len(result.inserted_ids)} employees into MongoDB")

inserted 10000 employees into MongoDB


In [48]:
#find many documents
def find_many_documents(query, projection=None):
    try:
        result = collection.find(query, projection)
        return result
    except Exception as e:
        print(f"error finding many document: {e}")
        return None

In [59]:
# logical operator is use because it matches documents
# that satisfies all the condition

query = { "$and": [
    {"performance_rating": {"$gte": 4.0}},
    {"salary": {"$gt": 80000}}
]
}
projection = {"_id": 0, "first_name": 1, "last_name": 1, "department": 1, "salary": 1, "performance rating": 1}
result = find_many_documents(query, projection).limit(5)
for doc in result:
        print(doc)

{'first_name': 'Brittany', 'last_name': 'Cain', 'department': 'Product', 'salary': 158098}
{'first_name': 'Kristie', 'last_name': 'Williams', 'department': 'Marketing', 'salary': 166255}
{'first_name': 'Joe', 'last_name': 'Bailey', 'department': 'Support', 'salary': 174256}
{'first_name': 'Peter', 'last_name': 'Turner', 'department': 'Sales', 'salary': 168474}
{'first_name': 'Dave', 'last_name': 'Holmes', 'department': 'Product', 'salary': 96145}


In [61]:
# logical operator is use because it matches documents
# that satisfies all the condition

query = { "$and": [
    {"years_experience": { "$gte": 5, "$lte": 10}},
    {"salary": {"$gt": 70000, "$lt": 120000}}
]
}
projection = {"_id": 0, "first_name": 1, "last_name": 1, "email": 1, "department": 1}
result = find_many_documents(query, projection).limit(5)
for doc in result:
    print(doc)

{'first_name': 'Chad', 'last_name': 'Herrera', 'email': 'leblanckevin@company.comexample.net', 'department': 'Support'}
{'first_name': 'Ryan', 'last_name': 'Fitzgerald', 'email': 'jimmyrobinson@company.comexample.net', 'department': 'Finance'}
{'first_name': 'Melissa', 'last_name': 'Miller', 'email': 'ryanchristine@company.comexample.com', 'department': 'Product'}
{'first_name': 'George', 'last_name': 'Pierce', 'email': 'allenalicia@company.comexample.net', 'department': 'Product'}
{'first_name': 'Juan', 'last_name': 'Brandt', 'email': 'sarah68@company.comexample.org', 'department': 'Finance'}


In [ ]:
# aggregate document
def aggregate_documents(pipeline):
    return collection.aggregate(pipeline)

In [62]:
# aggregation operators is used because of operators like $concate
pipeline = [
    {
     "$match": {
         "salary": {
             "$gte": 60000,
             "$lte": 100000
             }
         }
    },
    {
    "$project": {
        "_id": 0,
        "full_name":{"$concat": ["$first_name"," ",
"$last_name"]}, 
        "salary": 1,
        "years_experience": 1
        }
    },
    {
        "$limit": 5  # Limit to 5 results
    }
]

result = aggregate_documents(pipeline)
for doc in result:
    print(doc)

{'salary': 69083, 'years_experience': 19, 'full_name': 'Carol Abbott'}
{'salary': 85241, 'years_experience': 1, 'full_name': 'Kathryn Berry'}
{'salary': 61182, 'years_experience': 17, 'full_name': 'Jamie Wood'}
{'salary': 76520, 'years_experience': 20, 'full_name': 'Jennifer Moss'}
{'salary': 85020, 'years_experience': 1, 'full_name': 'Mary Roberts'}


In [63]:
# the datetime.strptime is use because this is the format that is accepted by mongodb

from datetime import datetime

pipeline = [
    {
        "$match": {
            "performance_rating": {"$gt": 3.5},
            "hire_date": {
                "$gte": datetime.strptime("2023-01-01T00:00:00.000+00:00", "%Y-%m-%dT%H:%M:%S.%f%z"),
                "$lte": datetime.strptime("2023-12-31T00:00:00.000+00:00", "%Y-%m-%dT%H:%M:%S.%f%z")
            }
        }        
    },
    {
    "$project":{
        "_id": 0,
        "full_name":{"$concat": ["$first_name"," ","$last_name"]},
        "salary": 1
        }
    },
    {
        "$limit": 5  # Limit to 5 results
    }
]

result = aggregate_documents(pipeline)
for doc in result:
    print(doc)

{'salary': 158098, 'full_name': 'Brittany Cain'}
{'salary': 96145, 'full_name': 'Dave Holmes'}
{'salary': 179295, 'full_name': 'Daniel Herrera'}
{'salary': 145583, 'full_name': 'Becky Patterson'}
{'salary': 164951, 'full_name': 'Holly Carter'}
